In [ ]:
pip install --upgrade azureml-sdk

**RESTART YOUR KERNEL**

## Workspace

Get the Azure ML workspace.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config('~/code/config.json')
ws

In [ ]:
ws.resource_group

In [ ]:
import os 

cmd = f'az network vnet create --name dask-vnet --resource-group {ws.resource_group} --subnet-name default'

os.system(cmd)

In [ ]:
import random
from azureml.core.compute import ComputeTarget, ComputeInstance

# Choose a name for your instance
# Compute instance name should be unique across the azure region
ci_name = f'ci-{str(random.randint(10000, 99999))}-dask'

compute_config = ComputeInstance.provisioning_configuration(
    vm_size='STANDARD_D3_V2',
    ssh_public_access=False,
    vnet_resourcegroup_name=ws.resource_group,
    vnet_name='dask-vnet',
    subnet_name='default',
)
instance = ComputeInstance.create(ws, ci_name, compute_config)
instance.wait_for_completion(show_output=True)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
ct_name = 'dask-vnet-ct'

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                       min_nodes=1,
                                                       max_nodes=100,
                                                       vnet_resourcegroup_name=ws.resource_group,
                                                       vnet_name='dask-vnet',
                                                       subnet_name='default'
)

ct = ComputeTarget.create(ws, ct_name, compute_config)
ct.wait_for_completion(show_output=True)

In [ ]:
instance_lab = f'https://{ci_name}.{ws.location}.instances.azureml.ms/lab'
print(instance_lab)